# Now we have a trained models,  lets take a list of keywords and find the business with the most similar reviews.


In [42]:
import pandas as pd
import pickle
import numpy as np
import pandas as pd
import pickle
import numpy as np

sys.path.append('../vectorsearch/')
import nltk_helper
import gensim

# Load the bar review dataset 
review = pd.read_pickle('../output/bar_reviews_cleaned_and_tokenized.pickle')
review.head(2)




model = gensim.models.Doc2Vec.load('../output/doc2vec_bars.model')



In [34]:

%autoreload 2 




def get_mean_word_vector(words, model):
    '''
    Given a dict of words and weights, determine the mean vector.
    words : dict
        Dict containing search terms and weights
    ''' 
    mean = np.zeros(model[words.keys()[0]].shape[0])
    vec = np.sum([model[word]*weight for (word, weight) in words.items()])
    return vec/np.dot(vec,vec)
    
def get_mean_doc_vector(review_ids, model):
    '''
    Given a list of review_ids, determine the mean vector.
    
    reviews : pandas_dataframe
    
    words : dict
        Dict containing search terms and weights
    ''' 
    # Indices of documents 
    # TODOL MAY NEED TO NORMALIZE DOC VECS 
    idx = np.array(model.docvecs.indexed_doctags(review_ids)[0])
    return np.sum(model.docvecs[idx], axis=0)
    
    
def dot_business(words, business_id, review):
    '''
    Find the mean cosine similarity over all reviews given a dict of words, and a business_id     
    
    words : dict
        Dict containing search terms and weights
    business_id : str
        str containing the business_id of interest.
        
    Returns
    ---------------
    similarity: float
        average cosine similarity of the word vector dotted onto the business reviews.  
    
    ''' 
    # Get the aggregate weighted word vector
    mean_words = get_mean_word_vector(words, model)
    # List the review ids associated with the given business
    review_ids = review_helper.get_review_ids(business_id, review)
    
    # Find similarity of the word vector and each review vector
    # First get the review indices in the doc2vec model 
    idx = np.array(model.docvecs.indexed_doctags(review_ids)[0])
    # Dot the business onto the mean words and average.
    # TODO: IS averaging the best?  Maybe we should take average over some threshold... 
    try:
        return np.max(np.dot(model.docvecs.doctag_syn0[idx], mean_words))
    except:
        print idx
business_ids = list(set(review.business_id.values[:10000]))
#print business_ids
cos_sim = [dot_business(words={'beer':10}, business_id=bus_id, review=review) for bus_id in business_ids]
print cos_sim
print np.argmax(cos_sim)
print review[business_ids[np.argmax(cos_sim)]==review.business_id].text.values

[0.03018349, 0.048298355, 0.047450565, 0.045716219, 0.034725033, 0.050144713, 0.048079018, 0.033967938, 0.044631425, 0.042930584, 0.042539414, 0.05046545, 0.049455069, 0.039392114, 0.031757724, 0.045100436, 0.039317604, 0.037079711, 0.037548412, 0.031666655, 0.036439285, 0.04620016, 0.044983082, 0.043383166, 0.042633962, 0.036718637, 0.038078286, 0.036916215, 0.049908012, 0.057226118, 0.036571726, 0.041692805, 0.039966062, 0.046239894, 0.051822498, 0.03698761, 0.038268931, 0.035325721, 0.049939983, 0.033832401, 0.031337637, 0.030860968, 0.03991903, 0.047244105, 0.044191163, 0.051531661, 0.040834837, 0.038522884, 0.029391523, 0.035072885, 0.030205453, 0.039120685, 0.03942303, 0.029587492, 0.045125455, 0.044605386, 0.029474536, 0.034709014, 0.040711265, 0.042947337, 0.041750092, 0.044095855, 0.037610024, 0.049462572, 0.055601545, 0.043686174, 0.043062862, 0.02967174, 0.044455636, 0.048898164, 0.048508283, 0.038926031, 0.034999616, 0.042092092, 0.039236132, 0.04528226, 0.039284952, 0.0343

In [33]:
[len(review.review_id[review.business_id==id]) for id in set(review.business_id.values[:100])]

[83, 4, 15, 26, 5, 5, 12, 10]

In [40]:
for word, sim in model.most_similar(positive=['beer', 'pricy']):
    print word, sim 

expensive 0.567374229431
pricey 0.560508370399
food 0.558115720749
inexpensive 0.546846270561
price 0.509109139442
steep 0.501822113991
pricier 0.484818696976
over-priced 0.483182609081
stiff 0.474921703339
average 0.468609929085


In [50]:

review.columns

Index([      u'business_id',              u'date',         u'review_id',
                   u'stars',              u'text',              u'type',
                 u'user_id',        u'votes_cool',       u'votes_funny',
            u'votes_useful', u'cleaned_tokenized'],
      dtype='object')

In [57]:
businesses = pd.read_pickle('../input/yelp_academic_dataset_business.pickle')


In [61]:
businesses = pd.read_pickle('../input/yelp_academic_dataset_business.pickle')

def GetBusinessIds(city='Las Vegas', categories=['Bars']):
    return (businesses[businesses['city']==city] & businesses['categories'].isin(categories))

In [62]:
print len(GetBusinessIds())

TypeError: unhashable type: 'list'